In [1]:

from pathlib import Path
from ReChisel.testcase import Testcase

bm = 'verilog-eval'

if bm == 'verilog-eval':
    bmcase = Testcase(
        'Prob030_popcount255', 
        'benchmarks/VerilogEval_sample/Prob030_popcount255_spec.txt',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_ref.sv',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_tb.sv'
    )
    output_dir = Path('output/VerilogEval/')
    output_dir.mkdir(parents=True, exist_ok=True)

elif bm == 'autochip':
    bmcase = Testcase(
        'Vector5',
        'benchmarks/AutoChip_sample/Vector5_spec.txt',
        None, # No reference file provided
        'benchmarks/AutoChip_sample/Vector5_0_tb.v'
    )
    output_dir = Path('output/AutoChip/')
    output_dir.mkdir(parents=True, exist_ok=True)


print(bmcase.specification)

I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in  (255 bits)
 - output out (  8 bits)

A "population count" circuit counts the number of '1's in an input
vector. The module should implement a population count circuit for a
255-bit input vector.


In [2]:

from rechisel import initial_chisel_generation
from ReChisel.chisel_code import ChiselCode


target_path = output_dir / 'gen.scala'
chisel_code: ChiselCode = initial_chisel_generation(bmcase, 'TopModule')
target_path.write_text(chisel_code.decorated)

print(chisel_code.decorated)


/home/niujuxin/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_arn" in BedrockRerank has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


package TopModule

import chisel3._
import chisel3.util._

import chisel3.stage.ChiselStage

class TopModule extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(255.W))  // 255-bit input vector
    val out = Output(UInt(8.W))   // 8-bit output for population count
  })

  // Population count logic using built-in method
  io.out := PopCount(io.in).asUInt() // Count number of '1's and convert to 8-bit
}

object Main extends App {
    (new ChiselStage).emitVerilog(
      new TopModule,
      Array(
        "--target-dir", "generated",
        "--emission-options=disableMemRandomization,disableRegisterRandomization",
      )
    )
}



In [3]:

from ReChisel.verifier import Verifier, VerifierWorkingSpace, VerifyResult


working_space = VerifierWorkingSpace(output_dir / 'chisel', output_dir / 'iv')
verifier = Verifier(working_space, bm)

is_ok = (
    verifier.prepare(chisel_code, bmcase) and
    verifier.compile() and
    verifier.iv() and
    verifier.vvp() and
    verifier.functionality()
)
verify_result: VerifyResult = verifier.result

is_ok

False

In [4]:
from pprint import pprint

pprint(verify_result.__dict__)

{'functionality_correct': False,
 'iv_error': '',
 'iv_success': False,
 'sbt_error': '[error] '
              '/home/niujuxin/github_repo/ReChisel/output/VerilogEval/chisel/src/main/scala/Main.scala:15:29: '
              'overloaded method apply with alternatives:\n'
              '[error]   (x: BigInt,y: BigInt)chisel3.UInt <and>\n'
              '[error]   (x: Int,y: Int)chisel3.UInt <and>\n'
              '[error]   (x: chisel3.UInt)chisel3.Bool <and>\n'
              '[error]   (x: Int)chisel3.Bool <and>\n'
              '[error]   (x: BigInt)chisel3.Bool\n'
              '[error]  cannot be applied to ()\n'
              '[error]   io.out := PopCount(io.in).asUInt() // Count number of '
              "'1's and convert to 8-bit\n"
              '[error]                             ^\n'
              '[error] one error found\n'
              '[error] (Compile / compileIncremental) Compilation failed\n'
              '[error] Total time: 1 s, completed 2025年6月29日 上午7:15:17',
 'sbt_